In [1]:
import numpy as np
import os
import pandas as pd
from pandas_profiling import ProfileReport
from scipy.stats import chi2

pd.options.display.float_format = '{:.4f}'.format

In [2]:
df_videos = pd.read_csv(os.path.join('..', 'data', 'AIcia_videos.csv'))
df_videos['公開日時'] = pd.to_datetime(df_videos['公開日時'])
df_videos['動画時間_s'] = pd.to_timedelta(df_videos['動画時間']).apply(lambda x: x.seconds)

df_videos = df_videos.drop(['動画時間'], axis=1)

df_videos.head()

,動画タイトル,公開日時,経過日数,プレミア公開,視聴回数,コメント,高評価件数,低評価件数,ln_経過日数,ln_視聴回数,ln_高評価件数,動画時間_s
0,【もう二度と迷わない】因子分析と主成分分析は何が違うの？【いろんな分析 vol. 3 】 #...,2020-04-30,4,1,714,18,69,0,1.6094,6.5723,4.2485,1082
1,【深層学習】学習 - なぜ必要なのか？何をするのか？【ディープラーニングの世界 vol. 2...,2020-04-28,6,1,1533,14,98,0,1.9459,7.3356,4.5951,1025
2,【相関で情報を圧縮】主成分分析の気持ちを理解する【いろんな分析 vol. 2 】 #051 ...,2020-04-23,11,1,1219,23,111,0,2.4849,7.1066,4.7185,1489
3,【深層学習】ディープラーニングとは関数近似器である【ディープラーニングの世界 vol. 1 ...,2020-04-21,13,1,2118,28,164,2,2.6391,7.6587,5.1059,530
4,【相関から構造を推定】因子分析の気持ちを理解する【いろんな分析 vol. 1 】#049 #...,2020-04-16,18,1,1462,18,112,1,2.9444,7.2882,4.7274,1283


In [3]:
profiling = ProfileReport(df_videos)

/Users/satoshi/Library/Python/3.7/lib/python/site-packages/matplotlib/textpath.py:90: RuntimeWarning: Glyph 32076 missing from current font.
  font.set_text(s, 0.0, flags=LOAD_NO_HINTING)
/Users/satoshi/Library/Python/3.7/lib/python/site-packages/matplotlib/textpath.py:90: RuntimeWarning: Glyph 36942 missing from current font.
  font.set_text(s, 0.0, flags=LOAD_NO_HINTING)
/Users/satoshi/Library/Python/3.7/lib/python/site-packages/matplotlib/textpath.py:90: RuntimeWarning: Glyph 26085 missing from current font.
  font.set_text(s, 0.0, flags=LOAD_NO_HINTING)
/Users/satoshi/Library/Python/3.7/lib/python/site-packages/matplotlib/textpath.py:90: RuntimeWarning: Glyph 25968 missing from current font.
  font.set_text(s, 0.0, flags=LOAD_NO_HINTING)
/Users/satoshi/Library/Python/3.7/lib/python/site-packages/matplotlib/textpath.py:203: RuntimeWarning: Glyph 32076 missing from current font.
  glyph = font.load_char(ccode, flags=LOAD_NO_HINTING)
/Users/satoshi/Library/Python/3.7/lib/python/site-p

/Users/satoshi/Library/Python/3.7/lib/python/site-packages/matplotlib/textpath.py:90: RuntimeWarning: Glyph 21205 missing from current font.
  font.set_text(s, 0.0, flags=LOAD_NO_HINTING)
/Users/satoshi/Library/Python/3.7/lib/python/site-packages/matplotlib/textpath.py:90: RuntimeWarning: Glyph 30011 missing from current font.
  font.set_text(s, 0.0, flags=LOAD_NO_HINTING)
/Users/satoshi/Library/Python/3.7/lib/python/site-packages/matplotlib/textpath.py:90: RuntimeWarning: Glyph 26178 missing from current font.
  font.set_text(s, 0.0, flags=LOAD_NO_HINTING)
/Users/satoshi/Library/Python/3.7/lib/python/site-packages/matplotlib/textpath.py:90: RuntimeWarning: Glyph 38291 missing from current font.
  font.set_text(s, 0.0, flags=LOAD_NO_HINTING)
/Users/satoshi/Library/Python/3.7/lib/python/site-packages/matplotlib/textpath.py:203: RuntimeWarning: Glyph 21205 missing from current font.
  glyph = font.load_char(ccode, flags=LOAD_NO_HINTING)
/Users/satoshi/Library/Python/3.7/lib/python/site-p

/Users/satoshi/Library/Python/3.7/lib/python/site-packages/matplotlib/textpath.py:90: RuntimeWarning: Glyph 12479 missing from current font.
  font.set_text(s, 0.0, flags=LOAD_NO_HINTING)
/Users/satoshi/Library/Python/3.7/lib/python/site-packages/matplotlib/textpath.py:90: RuntimeWarning: Glyph 12452 missing from current font.
  font.set_text(s, 0.0, flags=LOAD_NO_HINTING)
/Users/satoshi/Library/Python/3.7/lib/python/site-packages/matplotlib/textpath.py:90: RuntimeWarning: Glyph 12523 missing from current font.
  font.set_text(s, 0.0, flags=LOAD_NO_HINTING)
/Users/satoshi/Library/Python/3.7/lib/python/site-packages/matplotlib/textpath.py:90: RuntimeWarning: Glyph 20844 missing from current font.
  font.set_text(s, 0.0, flags=LOAD_NO_HINTING)
/Users/satoshi/Library/Python/3.7/lib/python/site-packages/matplotlib/textpath.py:90: RuntimeWarning: Glyph 38283 missing from current font.
  font.set_text(s, 0.0, flags=LOAD_NO_HINTING)
/Users/satoshi/Library/Python/3.7/lib/python/site-packages/m

In [4]:
profiling

# Implement Graphical Modeling

In [5]:
def inverse_cor(cor):
    cor_inv = np.linalg.inv(cor)
    diag = np.diag(1/np.sqrt(np.diag(cor_inv)))
    return diag.dot(cor_inv).dot(diag)

In [6]:
def estimate(sample_cor, cond_ind_pairs, error_torelance=1e-4, verbose=1):
    '''Estimate correlation matrix from sample correlation and pairs of indices with conditional independence

    Parameters
    ----------
    sample_cor : 2d-array
        sample correlation matric
    arg2 : list of pairs of int
        list of pairs of indices.
        if (i, j) is in the list, this means that the i-th and j-th variables are conditionally independent given all the other variables.
    error_torelance : float
        torelance of error
    verbose : int
        verbose
    
    Returns
    -------
    estimated_cor : 2d-array
        estimated correlation matrix
    
    '''
    dim = sample_cor.shape[0]
    sample_cor = np.array(sample_cor)
    
    estimated_cor = sample_cor.copy()    
    error = 1
    if verbose > 0:
        counter = 0
    
    while error > error_torelance:
        if verbose > 0:
            print('========', counter, '-th iteration', '========')
            counter = counter + 1
        
        estimated_cor_before = estimated_cor.copy()
        for i, j in cond_ind_pairs:
            estimated_cor_inv = np.linalg.inv(estimated_cor)
            new_ij = estimated_cor[i][j] + estimated_cor_inv[i][j]/(estimated_cor_inv[i][i] * estimated_cor_inv[j][j] - estimated_cor_inv[i][j]**2)
            estimated_cor[i][j] = new_ij
            estimated_cor[j][i] = new_ij
        
        error = np.abs(estimated_cor - estimated_cor_before).max().max()
    
    return estimated_cor

In [7]:
def deviance_and_p(original_cor, estimated_cor, df):
    dim = original_cor.shape[0]
    dev = dim * (np.log(np.linalg.det(estimated_cor)) - np.log(np.linalg.det(original_cor)))
    p = 1 - chi2.cdf(dev, df)
    
    return dev, p

# Selecting models

Model selection process:
1. Find the smallest (absolute) value in the standardized inverse of correlation matrix
2. Assume the pair of variables above are conditionally indepencent
3. Estimate correpation matrix with the assumption and see deviance and p-value.
4. If p >> 0, repeat the procedure
5. If p is small, then stop.

モデル選択は以下の順序で行う：
1. 相関行列の逆行列の標準化したものを計算し、（絶対）値最小のものを探す。
2. その2つの変数は、条件付き独立であると仮定する
3. その仮定のもと、相関行列を推定し、逸脱度とp-値を見る
4. p-値が大きければ、上記の手順を続行
5. p-値が小さければ終了。

In [8]:
df_videos_restricted = df_videos[['動画時間_s', '視聴回数', 'コメント', '高評価件数', '低評価件数']]
sample_cor = df_videos_restricted.corr()
column_names = df_videos_restricted.columns
estimated_cors = [sample_cor]

In [9]:
print('inverse of correlation')
pd.DataFrame(inverse_cor(estimated_cors[-1]), index=column_names, columns=column_names)

inverse of correlation


,動画時間_s,視聴回数,コメント,高評価件数,低評価件数
動画時間_s,1.0000,0.0577,-0.2602,-0.1509,0.1315
視聴回数,0.0577,1.0000,-0.4250,-0.7019,-0.7329
コメント,-0.2602,-0.4250,1.0000,-0.1184,0.1821
高評価件数,-0.1509,-0.7019,-0.1184,1.0000,0.3367
低評価件数,0.1315,-0.7329,0.1821,0.3367,1.0000


The correlation between `動画時間_s` and `視聴回数` attains the minimum, thus we assume those variables are conditionally independent.
Compute `estimated_correlation`, `deviance`, and `p`.

`動画時間_s` と `視聴回数` の相関が、絶対値最小なので、これらが条件付き独立だと仮定。
相関行列の推定値と、逸脱度と、p-値を計算する。

In [10]:

cond_ind_pairs = [(0, 1)]

estimated_cors.append(estimate(estimated_cors[-1], cond_ind_pairs))

print('========', 'original', '========')
df_original = pd.DataFrame(estimated_cors[0], index=column_names, columns=column_names)
display(df_original)

print('========', 'estimated', '========')
df_estimated = pd.DataFrame(estimated_cors[-1], index=column_names, columns=column_names)
display(df_estimated)


print('========', 'test rel to original', '========')
dev, p = deviance_and_p(estimated_cors[0], estimated_cors[-1], df=len(cond_ind_pairs))
print('dev:', dev)
print('p:', p)

print('========', 'test rel to prev', '========')
dev, p = deviance_and_p(estimated_cors[-2], estimated_cors[-1], df=1)
print('dev:', dev)
print('p:', p)

======== 0 -th iteration ========
======== 1 -th iteration ========
======== original ========


,動画時間_s,視聴回数,コメント,高評価件数,低評価件数
動画時間_s,1.0000,0.3289,0.4521,0.4101,0.1251
視聴回数,0.3289,1.0000,0.8486,0.9104,0.8311
コメント,0.4521,0.8486,1.0000,0.8322,0.6137
高評価件数,0.4101,0.9104,0.8322,1.0000,0.6606
低評価件数,0.1251,0.8311,0.6137,0.6606,1.0000


======== estimated ========


,動画時間_s,視聴回数,コメント,高評価件数,低評価件数
動画時間_s,1.0000,0.3414,0.4521,0.4101,0.1251
視聴回数,0.3414,1.0000,0.8486,0.9104,0.8311
コメント,0.4521,0.8486,1.0000,0.8322,0.6137
高評価件数,0.4101,0.9104,0.8322,1.0000,0.6606
低評価件数,0.1251,0.8311,0.6137,0.6606,1.0000


======== test rel to original ========
dev: 0.01664772806781123
p: 0.8973370162960576
======== test rel to prev ========
dev: 0.01664772806781123
p: 0.8973370162960576


The value of p is enough large -> continue

p-値が充分大きいので、続行。

In [11]:
print('inverse of correlation')
pd.DataFrame(inverse_cor(estimated_cors[-1]), index=column_names, columns=column_names)

inverse of correlation


,動画時間_s,視聴回数,コメント,高評価件数,低評価件数
動画時間_s,1.0000,0.0000,-0.2375,-0.1112,0.1729
視聴回数,0.0000,1.0000,-0.4131,-0.6980,-0.7369
コメント,-0.2375,-0.4131,1.0000,-0.1330,0.1751
高評価件数,-0.1112,-0.6980,-0.1330,1.0000,0.3369
低評価件数,0.1729,-0.7369,0.1751,0.3369,1.0000


In [12]:
cond_ind_pairs = [(0, 1), (0, 3)]  # the correlation between `動画時間_s`　and `高評価件数` is the smallest

estimated_cors.append(estimate(estimated_cors[-1], cond_ind_pairs))

print('========', 'original', '========')
df_original = pd.DataFrame(estimated_cors[0], index=column_names, columns=column_names)
display(df_original)

print('========', 'estimated', '========')
df_estimated = pd.DataFrame(estimated_cors[-1], index=column_names, columns=column_names)
display(df_estimated)


print('========', 'test rel to original', '========')
dev, p = deviance_and_p(estimated_cors[0], estimated_cors[-1], df=len(cond_ind_pairs))
print('dev:', dev)
print('p:', p)

print('========', 'test rel to prev', '========')
dev, p = deviance_and_p(estimated_cors[-2], estimated_cors[-1], df=1)
print('dev:', dev)
print('p:', p)

======== 0 -th iteration ========
======== 1 -th iteration ========
======== 2 -th iteration ========
======== 3 -th iteration ========
======== 4 -th iteration ========
======== 5 -th iteration ========
======== 6 -th iteration ========
======== 7 -th iteration ========
======== 8 -th iteration ========
======== 9 -th iteration ========
======== original ========


,動画時間_s,視聴回数,コメント,高評価件数,低評価件数
動画時間_s,1.0000,0.3289,0.4521,0.4101,0.1251
視聴回数,0.3289,1.0000,0.8486,0.9104,0.8311
コメント,0.4521,0.8486,1.0000,0.8322,0.6137
高評価件数,0.4101,0.9104,0.8322,1.0000,0.6606
低評価件数,0.1251,0.8311,0.6137,0.6606,1.0000


======== estimated ========


,動画時間_s,視聴回数,コメント,高評価件数,低評価件数
動画時間_s,1.0000,0.3079,0.4521,0.3397,0.1251
視聴回数,0.3079,1.0000,0.8486,0.9104,0.8311
コメント,0.4521,0.8486,1.0000,0.8322,0.6137
高評価件数,0.3397,0.9104,0.8322,1.0000,0.6606
低評価件数,0.1251,0.8311,0.6137,0.6606,1.0000


======== test rel to original ========
dev: 0.1387798139653329
p: 0.9329628404249507
======== test rel to prev ========
dev: 0.12213208589752167
p: 0.7267334769115062


In [13]:
print('inverse of correlation')
pd.DataFrame(inverse_cor(estimated_cors[-1]), index=column_names, columns=column_names)

inverse of correlation


,動画時間_s,視聴回数,コメント,高評価件数,低評価件数
動画時間_s,1.0000,-0.0001,-0.3242,0.0000,0.1370
視聴回数,-0.0001,1.0000,-0.4023,-0.7024,-0.7412
コメント,-0.3242,-0.4023,1.0000,-0.1562,0.1674
高評価件数,0.0000,-0.7024,-0.1562,1.0000,0.3604
低評価件数,0.1370,-0.7412,0.1674,0.3604,1.0000


In [14]:
cond_ind_pairs = [(0, 1), (0, 3), (0, 4)]  # the correlation between `動画時間_s`　and `低評価件数` is the smallest

estimated_cors.append(estimate(estimated_cors[-1], cond_ind_pairs))

print('========', 'original', '========')
df_original = pd.DataFrame(estimated_cors[0], index=column_names, columns=column_names)
display(df_original)

print('========', 'estimated', '========')
df_estimated = pd.DataFrame(estimated_cors[-1], index=column_names, columns=column_names)
display(df_estimated)


print('========', 'test rel to original', '========')
dev, p = deviance_and_p(estimated_cors[0], estimated_cors[-1], df=len(cond_ind_pairs))
print('dev:', dev)
print('p:', p)

print('========', 'test rel to prev', '========')
dev, p = deviance_and_p(estimated_cors[-2], estimated_cors[-1], df=1)
print('dev:', dev)
print('p:', p)

======== 0 -th iteration ========
======== 1 -th iteration ========
======== 2 -th iteration ========
======== 3 -th iteration ========
======== 4 -th iteration ========
======== 5 -th iteration ========
======== 6 -th iteration ========
======== 7 -th iteration ========
======== 8 -th iteration ========
======== 9 -th iteration ========
======== 10 -th iteration ========
======== 11 -th iteration ========
======== 12 -th iteration ========
======== 13 -th iteration ========
======== 14 -th iteration ========
======== 15 -th iteration ========
======== 16 -th iteration ========
======== 17 -th iteration ========
======== 18 -th iteration ========
======== 19 -th iteration ========
======== original ========


,動画時間_s,視聴回数,コメント,高評価件数,低評価件数
動画時間_s,1.0000,0.3289,0.4521,0.4101,0.1251
視聴回数,0.3289,1.0000,0.8486,0.9104,0.8311
コメント,0.4521,0.8486,1.0000,0.8322,0.6137
高評価件数,0.4101,0.9104,0.8322,1.0000,0.6606
低評価件数,0.1251,0.8311,0.6137,0.6606,1.0000


======== estimated ========


,動画時間_s,視聴回数,コメント,高評価件数,低評価件数
動画時間_s,1.0000,0.3835,0.4521,0.3761,0.2772
視聴回数,0.3835,1.0000,0.8486,0.9104,0.8311
コメント,0.4521,0.8486,1.0000,0.8322,0.6137
高評価件数,0.3761,0.9104,0.8322,1.0000,0.6606
低評価件数,0.2772,0.8311,0.6137,0.6606,1.0000


======== test rel to original ========
dev: 0.3785010920895804
p: 0.9446469677397838
======== test rel to prev ========
dev: 0.23972127812424748
p: 0.6244074948766589


In [15]:
print('inverse of correlation')
pd.DataFrame(inverse_cor(estimated_cors[-1]), index=column_names, columns=column_names)

inverse of correlation


,動画時間_s,視聴回数,コメント,高評価件数,低評価件数
動画時間_s,1.0000,0.0002,-0.2440,-0.0001,-0.0000
視聴回数,0.0002,1.0000,-0.4125,-0.7024,-0.7482
コメント,-0.2440,-0.4125,1.0000,-0.1602,0.2192
高評価件数,-0.0001,-0.7024,-0.1602,1.0000,0.3639
低評価件数,-0.0000,-0.7482,0.2192,0.3639,1.0000


In [16]:
cond_ind_pairs = [(0, 1), (0, 3), (0, 4), (2, 3)]

estimated_cors.append(estimate(estimated_cors[-1], cond_ind_pairs))

print('========', 'original', '========')
df_original = pd.DataFrame(estimated_cors[0], index=column_names, columns=column_names)
display(df_original)

print('========', 'estimated', '========')
df_estimated = pd.DataFrame(estimated_cors[-1], index=column_names, columns=column_names)
display(df_estimated)


print('========', 'test rel to original', '========')
dev, p = deviance_and_p(estimated_cors[0], estimated_cors[-1], df=len(cond_ind_pairs))
print('dev:', dev)
print('p:', p)

print('========', 'test rel to prev', '========')
dev, p = deviance_and_p(estimated_cors[-2], estimated_cors[-1], df=1)
print('dev:', dev)
print('p:', p)

======== 0 -th iteration ========
======== 1 -th iteration ========
======== 2 -th iteration ========
======== 3 -th iteration ========
======== 4 -th iteration ========
======== 5 -th iteration ========
======== 6 -th iteration ========
======== 7 -th iteration ========
======== 8 -th iteration ========
======== 9 -th iteration ========
======== 10 -th iteration ========
======== 11 -th iteration ========
======== 12 -th iteration ========
======== 13 -th iteration ========
======== 14 -th iteration ========
======== original ========


,動画時間_s,視聴回数,コメント,高評価件数,低評価件数
動画時間_s,1.0000,0.3289,0.4521,0.4101,0.1251
視聴回数,0.3289,1.0000,0.8486,0.9104,0.8311
コメント,0.4521,0.8486,1.0000,0.8322,0.6137
高評価件数,0.4101,0.9104,0.8322,1.0000,0.6606
低評価件数,0.1251,0.8311,0.6137,0.6606,1.0000


======== estimated ========


,動画時間_s,視聴回数,コメント,高評価件数,低評価件数
動画時間_s,1.0000,0.3839,0.4521,0.3623,0.2777
視聴回数,0.3839,1.0000,0.8486,0.9104,0.8311
コメント,0.4521,0.8486,1.0000,0.8010,0.6137
高評価件数,0.3623,0.9104,0.8010,1.0000,0.6606
低評価件数,0.2777,0.8311,0.6137,0.6606,1.0000


======== test rel to original ========
dev: 0.5167903076315739
p: 0.9718459914074907
======== test rel to prev ========
dev: 0.13828921554199347
p: 0.7099877216064581


In [17]:
print('inverse of correlation')
pd.DataFrame(inverse_cor(estimated_cors[-1]), index=column_names, columns=column_names)

inverse of correlation


,動画時間_s,視聴回数,コメント,高評価件数,低評価件数
動画時間_s,1.0000,-0.0003,-0.2466,0.0001,0.0000
視聴回数,-0.0003,1.0000,-0.5025,-0.7401,-0.7541
コメント,-0.2466,-0.5025,1.0000,-0.0000,0.2766
高評価件数,0.0001,-0.7401,-0.0000,1.0000,0.4002
低評価件数,0.0000,-0.7541,0.2766,0.4002,1.0000


In [18]:
cond_ind_pairs = [(0, 1), (0, 3), (0, 4), (2, 3), (0, 2)]

estimated_cors.append(estimate(estimated_cors[-1], cond_ind_pairs))

print('========', 'original', '========')
df_original = pd.DataFrame(estimated_cors[0], index=column_names, columns=column_names)
display(df_original)

print('========', 'estimated', '========')
df_estimated = pd.DataFrame(estimated_cors[-1], index=column_names, columns=column_names)
display(df_estimated)


print('========', 'test rel to original', '========')
dev, p = deviance_and_p(estimated_cors[0], estimated_cors[-1], df=len(cond_ind_pairs))
print('dev:', dev)
print('p:', p)

print('========', 'test rel to prev', '========')
dev, p = deviance_and_p(estimated_cors[-2], estimated_cors[-1], df=1)
print('dev:', dev)
print('p:', p)

======== 0 -th iteration ========
======== 1 -th iteration ========
======== 2 -th iteration ========
======== 3 -th iteration ========
======== 4 -th iteration ========
======== 5 -th iteration ========
======== 6 -th iteration ========
======== 7 -th iteration ========
======== 8 -th iteration ========
======== 9 -th iteration ========
======== 10 -th iteration ========
======== 11 -th iteration ========
======== 12 -th iteration ========
======== 13 -th iteration ========
======== 14 -th iteration ========
======== 15 -th iteration ========
======== 16 -th iteration ========
======== 17 -th iteration ========
======== 18 -th iteration ========
======== 19 -th iteration ========
======== 20 -th iteration ========
======== 21 -th iteration ========
======== 22 -th iteration ========
======== 23 -th iteration ========
======== 24 -th iteration ========
======== 25 -th iteration ========
======== 26 -th iteration ========
======== 27 -th iteration ========
======== 28 -th iteration ====

,動画時間_s,視聴回数,コメント,高評価件数,低評価件数
動画時間_s,1.0000,0.3289,0.4521,0.4101,0.1251
視聴回数,0.3289,1.0000,0.8486,0.9104,0.8311
コメント,0.4521,0.8486,1.0000,0.8322,0.6137
高評価件数,0.4101,0.9104,0.8322,1.0000,0.6606
低評価件数,0.1251,0.8311,0.6137,0.6606,1.0000


======== estimated ========


,動画時間_s,視聴回数,コメント,高評価件数,低評価件数
動画時間_s,1.0000,0.0013,0.0011,0.0012,0.0011
視聴回数,0.0013,1.0000,0.8486,0.9104,0.8311
コメント,0.0011,0.8486,1.0000,0.8010,0.6137
高評価件数,0.0012,0.9104,0.8010,1.0000,0.6606
低評価件数,0.0011,0.8311,0.6137,0.6606,1.0000


======== test rel to original ========
dev: 1.660048649578214
p: 0.8938940073341245
======== test rel to prev ========
dev: 1.1432583419466402
p: 0.28496487482957844


It seems that it is not better idea to impose the conditional independence between 0 and 2 since with the condition, 0-th variable is independent with all the other variables.

0 と 2 の条件付き独立を仮定すると、 0 が他のすべての変数と独立になるので、良くないらしい。

In [19]:
cond_ind_pairs = [(0, 1), (0, 3), (0, 4), (2, 3)]  # remove (0, 2)

estimated_cors.append(estimate(estimated_cors[-2], cond_ind_pairs))  # -1 -> -2

print('========', 'original', '========')
df_original = pd.DataFrame(estimated_cors[0], index=column_names, columns=column_names)
display(df_original)

print('========', 'estimated', '========')
df_estimated = pd.DataFrame(estimated_cors[-1], index=column_names, columns=column_names)
display(df_estimated)


print('========', 'test rel to original', '========')
dev, p = deviance_and_p(estimated_cors[0], estimated_cors[-1], df=len(cond_ind_pairs))
print('dev:', dev)
print('p:', p)

print('========', 'test rel to prev', '========')
dev, p = deviance_and_p(estimated_cors[-2], estimated_cors[-1], df=1)
print('dev:', dev)
print('p:', p)

======== 0 -th iteration ========
======== original ========


,動画時間_s,視聴回数,コメント,高評価件数,低評価件数
動画時間_s,1.0000,0.3289,0.4521,0.4101,0.1251
視聴回数,0.3289,1.0000,0.8486,0.9104,0.8311
コメント,0.4521,0.8486,1.0000,0.8322,0.6137
高評価件数,0.4101,0.9104,0.8322,1.0000,0.6606
低評価件数,0.1251,0.8311,0.6137,0.6606,1.0000


======== estimated ========


,動画時間_s,視聴回数,コメント,高評価件数,低評価件数
動画時間_s,1.0000,0.3838,0.4521,0.3623,0.2777
視聴回数,0.3838,1.0000,0.8486,0.9104,0.8311
コメント,0.4521,0.8486,1.0000,0.8010,0.6137
高評価件数,0.3623,0.9104,0.8010,1.0000,0.6606
低評価件数,0.2777,0.8311,0.6137,0.6606,1.0000


======== test rel to original ========
dev: 0.5167908589276937
p: 0.9718459364002466
======== test rel to prev ========
dev: -1.1432577906505204
p: 1.0


In [20]:
print('inverse of correlation')
pd.DataFrame(inverse_cor(estimated_cors[-1]), index=column_names, columns=column_names)

inverse of correlation


,動画時間_s,視聴回数,コメント,高評価件数,低評価件数
動画時間_s,1.0000,-0.0002,-0.2467,0.0001,0.0000
視聴回数,-0.0002,1.0000,-0.5025,-0.7401,-0.7541
コメント,-0.2467,-0.5025,1.0000,0.0000,0.2766
高評価件数,0.0001,-0.7401,0.0000,1.0000,0.4002
低評価件数,0.0000,-0.7541,0.2766,0.4002,1.0000


In [21]:
cond_ind_pairs = [(0, 1), (0, 3), (0, 4), (2, 3), (2, 4)]

estimated_cors.append(estimate(estimated_cors[-1], cond_ind_pairs))

print('========', 'original', '========')
df_original = pd.DataFrame(estimated_cors[0], index=column_names, columns=column_names)
display(df_original)

print('========', 'estimated', '========')
df_estimated = pd.DataFrame(estimated_cors[-1], index=column_names, columns=column_names)
display(df_estimated)


print('========', 'test rel to original', '========')
dev, p = deviance_and_p(estimated_cors[0], estimated_cors[-1], df=len(cond_ind_pairs))
print('dev:', dev)
print('p:', p)

print('========', 'test rel to prev', '========')
dev, p = deviance_and_p(estimated_cors[-2], estimated_cors[-1], df=1)
print('dev:', dev)
print('p:', p)

======== 0 -th iteration ========
======== 1 -th iteration ========
======== 2 -th iteration ========
======== 3 -th iteration ========
======== 4 -th iteration ========
======== 5 -th iteration ========
======== 6 -th iteration ========
======== 7 -th iteration ========
======== 8 -th iteration ========
======== 9 -th iteration ========
======== 10 -th iteration ========
======== 11 -th iteration ========
======== original ========


,動画時間_s,視聴回数,コメント,高評価件数,低評価件数
動画時間_s,1.0000,0.3289,0.4521,0.4101,0.1251
視聴回数,0.3289,1.0000,0.8486,0.9104,0.8311
コメント,0.4521,0.8486,1.0000,0.8322,0.6137
高評価件数,0.4101,0.9104,0.8322,1.0000,0.6606
低評価件数,0.1251,0.8311,0.6137,0.6606,1.0000


======== estimated ========


,動画時間_s,視聴回数,コメント,高評価件数,低評価件数
動画時間_s,1.0000,0.3839,0.4521,0.3494,0.3191
視聴回数,0.3839,1.0000,0.8486,0.9104,0.8311
コメント,0.4521,0.8486,1.0000,0.7725,0.7053
高評価件数,0.3494,0.9104,0.7725,1.0000,0.6606
低評価件数,0.3191,0.8311,0.7053,0.6606,1.0000


======== test rel to original ========
dev: 1.0268339446265617
p: 0.9603725220359802
======== test rel to prev ========
dev: 0.510043085698868
p: 0.4751202052097019


Not so bad.

こちらは悪くない。

In [22]:
print('inverse of correlation')
pd.DataFrame(inverse_cor(estimated_cors[-1]), index=column_names, columns=column_names)

inverse of correlation


,動画時間_s,視聴回数,コメント,高評価件数,低評価件数
動画時間_s,1.0000,-0.0003,-0.2586,0.0001,0.0000
視聴回数,-0.0003,1.0000,-0.3939,-0.7903,-0.6753
コメント,-0.2586,-0.3939,1.0000,0.0000,0.0000
高評価件数,0.0001,-0.7903,0.0000,1.0000,0.4176
低評価件数,0.0000,-0.6753,0.0000,0.4176,1.0000


In [23]:
cond_ind_pairs = [(0, 1), (0, 3), (0, 4), (2, 3), (2, 4), (1, 2)]

estimated_cors.append(estimate(estimated_cors[-1], cond_ind_pairs))

print('========', 'original', '========')
df_original = pd.DataFrame(estimated_cors[0], index=column_names, columns=column_names)
display(df_original)

print('========', 'estimated', '========')
df_estimated = pd.DataFrame(estimated_cors[-1], index=column_names, columns=column_names)
display(df_estimated)


print('========', 'test rel to original', '========')
dev, p = deviance_and_p(estimated_cors[0], estimated_cors[-1], df=len(cond_ind_pairs))
print('dev:', dev)
print('p:', p)

print('========', 'test rel to prev', '========')
dev, p = deviance_and_p(estimated_cors[-2], estimated_cors[-1], df=1)
print('dev:', dev)
print('p:', p)

======== 0 -th iteration ========
======== 1 -th iteration ========
======== 2 -th iteration ========
======== 3 -th iteration ========
======== 4 -th iteration ========
======== 5 -th iteration ========
======== 6 -th iteration ========
======== 7 -th iteration ========
======== 8 -th iteration ========
======== 9 -th iteration ========
======== 10 -th iteration ========
======== 11 -th iteration ========
======== 12 -th iteration ========
======== 13 -th iteration ========
======== 14 -th iteration ========
======== 15 -th iteration ========
======== 16 -th iteration ========
======== 17 -th iteration ========
======== 18 -th iteration ========
======== 19 -th iteration ========
======== 20 -th iteration ========
======== 21 -th iteration ========
======== 22 -th iteration ========
======== 23 -th iteration ========
======== 24 -th iteration ========
======== 25 -th iteration ========
======== 26 -th iteration ========
======== 27 -th iteration ========
======== 28 -th iteration ====

,動画時間_s,視聴回数,コメント,高評価件数,低評価件数
動画時間_s,1.0000,0.3289,0.4521,0.4101,0.1251
視聴回数,0.3289,1.0000,0.8486,0.9104,0.8311
コメント,0.4521,0.8486,1.0000,0.8322,0.6137
高評価件数,0.4101,0.9104,0.8322,1.0000,0.6606
低評価件数,0.1251,0.8311,0.6137,0.6606,1.0000


======== estimated ========


,動画時間_s,視聴回数,コメント,高評価件数,低評価件数
動画時間_s,1.0000,0.0019,0.4521,0.0017,0.0017
視聴回数,0.0019,1.0000,0.0018,0.9104,0.8311
コメント,0.4521,0.0018,1.0000,0.0017,0.0017
高評価件数,0.0017,0.9104,0.0017,1.0000,0.6606
低評価件数,0.0017,0.8311,0.0017,0.6606,1.0000


======== test rel to original ========
dev: 7.393823963799693
p: 0.2859560770291275
======== test rel to prev ========
dev: 6.366990019173131
p: 0.011626263572538154


The p-value is too small.
Thus, the list of conditional independence is `[(0, 1), (0, 3), (0, 4), (2, 3), (2, 4)]`

p-値が小さすぎる。
なので、条件付き独立な変数のリストは `[(0, 1), (0, 3), (0, 4), (2, 3), (2, 4)]` だろう。

In [24]:
# sample correlation
print('sample correlation', '標本相関係数')
display(pd.DataFrame(estimated_cors[0], index=column_names, columns=column_names))

# estimated correlation
print('estimated correlation', '相関係数の推定値')
display(pd.DataFrame(estimated_cors[-2], index=column_names, columns=column_names))

# estimated vs sample
print('estimated vs sample', '値の差')
display(pd.DataFrame(estimated_cors[-2] - estimated_cors[0], index=column_names, columns=column_names))

# compute partial correlation
print('partial correlation', '偏相関係数')
display(pd.DataFrame(inverse_cor(estimated_cors[-2]), index=column_names, columns=column_names).applymap(lambda x: -x if x < 1-1e-4 else 1))

sample correlation 標本相関係数


,動画時間_s,視聴回数,コメント,高評価件数,低評価件数
動画時間_s,1.0000,0.3289,0.4521,0.4101,0.1251
視聴回数,0.3289,1.0000,0.8486,0.9104,0.8311
コメント,0.4521,0.8486,1.0000,0.8322,0.6137
高評価件数,0.4101,0.9104,0.8322,1.0000,0.6606
低評価件数,0.1251,0.8311,0.6137,0.6606,1.0000


estimated correlation 相関係数の推定値


,動画時間_s,視聴回数,コメント,高評価件数,低評価件数
動画時間_s,1.0000,0.3839,0.4521,0.3494,0.3191
視聴回数,0.3839,1.0000,0.8486,0.9104,0.8311
コメント,0.4521,0.8486,1.0000,0.7725,0.7053
高評価件数,0.3494,0.9104,0.7725,1.0000,0.6606
低評価件数,0.3191,0.8311,0.7053,0.6606,1.0000


estimated vs sample 値の差


,動画時間_s,視聴回数,コメント,高評価件数,低評価件数
動画時間_s,0.0000,0.0549,0.0000,-0.0606,0.1940
視聴回数,0.0549,0.0000,0.0000,0.0000,0.0000
コメント,0.0000,0.0000,0.0000,-0.0597,0.0916
高評価件数,-0.0606,0.0000,-0.0597,0.0000,0.0000
低評価件数,0.1940,0.0000,0.0916,0.0000,0.0000


partial correlation 偏相関係数


,動画時間_s,視聴回数,コメント,高評価件数,低評価件数
動画時間_s,1.0000,0.0003,0.2586,-0.0001,-0.0000
視聴回数,0.0003,1.0000,0.3939,0.7903,0.6753
コメント,0.2586,0.3939,1.0000,-0.0000,-0.0000
高評価件数,-0.0001,0.7903,-0.0000,1.0000,-0.4176
低評価件数,-0.0000,0.6753,-0.0000,-0.4176,1.0000


Thus, we have the following graphical model.

このようにして、次のグラフィカルモデルを手に入れた。

<img src='figure/GraphicalModel.png'>